In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
# Train data read and split
train_data = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
train_labels = train_data.pop('Transported')

In [3]:
# Function to divide numerical and catogorical columns
def divide_cols(df) :
    num_types = np.array(["int","float"])
    nums = df.select_dtypes(num_types).columns
    cats = df.select_dtypes(exclude=num_types).columns
    return nums,cats

In [4]:
cabin_splits = train_data[train_data.Cabin.notna()]["Cabin"].str.split("/",n=2,expand=True)
train_data[["CabinDeck","CabinNum","CabinSide"]] = cabin_splits
train_data.drop(columns=["Cabin","CabinNum","Name"],inplace=True)

In [5]:
nums,cats = divide_cols(train_data)
cats = cats.drop("PassengerId")

In [6]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [7]:
iterative_imputer = IterativeImputer(max_iter=10, random_state=99)
train_data[nums] = pd.DataFrame(iterative_imputer.fit_transform(train_data[nums]), columns= nums)

In [8]:
train_data["Bill"] = train_data["RoomService"]+train_data["FoodCourt"]+train_data["ShoppingMall"]+train_data["Spa"]+train_data["VRDeck"]
train_data.drop(columns=["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"],inplace=True)

In [9]:
from sklearn.preprocessing import KBinsDiscretizer
kbd = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')

In [10]:
nums,cats = divide_cols(train_data)
cats = cats.drop("PassengerId")
train_data[nums] = pd.DataFrame(kbd.fit_transform(train_data[nums]),columns=nums)

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_discretization.py:233: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  "decreasing the number of bins." % jj


In [11]:
from sklearn.impute import SimpleImputer
categorical_imputer = SimpleImputer(strategy= "most_frequent")
train_data[cats] = pd.DataFrame(categorical_imputer.fit_transform(train_data[cats]), columns= cats)

In [12]:
cats

Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'CabinDeck',
       'CabinSide'],
      dtype='object')

In [13]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
train_data[cats] = encoder.fit_transform(train_data[cats])
train_data.drop("PassengerId", axis= 1, inplace= True)

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_data, train_labels, train_size = 0.8,stratify=train_labels)

In [15]:
X_train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,CabinDeck,CabinSide,Bill
7979,2.0,1.0,2.0,4.0,0.0,5.0,1.0,0.0
8220,0.0,0.0,2.0,0.0,0.0,6.0,0.0,0.0
4220,0.0,1.0,2.0,3.0,0.0,6.0,0.0,0.0
7662,1.0,0.0,2.0,4.0,0.0,2.0,1.0,3.0
4328,0.0,1.0,2.0,1.0,0.0,6.0,1.0,0.0


In [16]:
from xgboost import XGBClassifier
clf = XGBClassifier(n_estimators=1500, learning_rate=0.01, early_stopping_rounds=20, objective="binary:logistic")
clf.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])

[0]	validation_0-logloss:0.69006
[1]	validation_0-logloss:0.68702
[2]	validation_0-logloss:0.68406
[3]	validation_0-logloss:0.68114
[4]	validation_0-logloss:0.67828
[5]	validation_0-logloss:0.67548
[6]	validation_0-logloss:0.67273
[7]	validation_0-logloss:0.67004
[8]	validation_0-logloss:0.66740
[9]	validation_0-logloss:0.66481
[10]	validation_0-logloss:0.66227
[11]	validation_0-logloss:0.65978
[12]	validation_0-logloss:0.65734
[13]	validation_0-logloss:0.65493
[14]	validation_0-logloss:0.65259
[15]	validation_0-logloss:0.65028
[16]	validation_0-logloss:0.64801
[17]	validation_0-logloss:0.64579
[18]	validation_0-logloss:0.64360
[19]	validation_0-logloss:0.64144
[20]	validation_0-logloss:0.63934
[21]	validation_0-logloss:0.63726
[22]	validation_0-logloss:0.63523
[23]	validation_0-logloss:0.63322
[24]	validation_0-logloss:0.63127
[25]	validation_0-logloss:0.62934
[26]	validation_0-logloss:0.62745
[27]	validation_0-logloss:0.62558
[28]	validation_0-logloss:0.62375
[29]	validation_0-loglos

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=20, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.01, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=1500,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [17]:
test_data = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")

In [18]:
cabin_splits = test_data[test_data.Cabin.notna()]["Cabin"].str.split("/",n=2,expand=True)
test_data[["CabinDeck","CabinNum","CabinSide"]] = cabin_splits
test_data.drop(columns=["Cabin","CabinNum","Name"],inplace=True)
nums,cats = divide_cols(test_data)
cats = cats.drop(["PassengerId"])

In [19]:
test_data[nums] = pd.DataFrame(iterative_imputer.transform(test_data[nums]), columns= nums)

In [20]:
test_data["Bill"] = test_data["RoomService"]+test_data["FoodCourt"]+test_data["ShoppingMall"]+test_data["Spa"]+test_data["VRDeck"]
test_data.drop(columns=["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"],inplace=True)

In [21]:
nums,cats = divide_cols(test_data)
cats = cats.drop(["PassengerId"])
test_data[nums] = pd.DataFrame(kbd.transform(test_data[nums]),columns=nums)
test_data[cats] = pd.DataFrame(categorical_imputer.fit_transform(test_data[cats]), columns= cats)
test_data[cats] = encoder.transform(test_data[cats])

In [22]:
test_data.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,CabinDeck,CabinSide,Bill
0,0013_01,0.0,1.0,2.0,2.0,0.0,6.0,1.0,0.0
1,0018_01,0.0,0.0,2.0,1.0,0.0,5.0,1.0,3.0
2,0019_01,1.0,1.0,0.0,3.0,0.0,2.0,1.0,0.0
3,0021_01,1.0,0.0,2.0,3.0,0.0,2.0,1.0,3.0
4,0023_01,0.0,0.0,2.0,1.0,0.0,5.0,1.0,1.0


In [23]:
preds = [x == 1 for x in clf.predict(test_data[train_data.columns])]


In [24]:
submission_df = pd.DataFrame({
    "PassengerId" : test_data["PassengerId"],
    "Transported" : preds
})
submission_df.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False


In [25]:
submission_df.to_csv("submission.csv", index=False)